## Madmom (ran with the madmom mamba env)

#### BLAH BLAH

In [2]:
%matplotlib inline

import madmom
import numpy as np
from pathlib import Path
from librosa import clicks
import matplotlib.pyplot as plt
from IPython.display import Audio

In [3]:
N = 95
songPath = Path(f"~/Documents/theSalsaProject/audioFragments/{N}.wav").expanduser()
sig = madmom.audio.signal.Signal(data = songPath)
sig

Signal([  -1,    2,   -3, ..., -770, -876, -814], dtype=int16)

In [163]:
Audio(sig.T, rate = sig.sample_rate)

In [47]:
rnn = madmom.features.beats.RNNBeatProcessor()
btp = madmom.features.beats.DBNBeatTrackingProcessor(fps = 100)
res = rnn(sig) 
oddBeats = btp(res)

#### blah blah

In [ ]:
import madmom
from madmom.models import BEATS_LSTM, BEATS_BLSTM

import pickle
import numpy as np

import torch
from torch import nn
from madmom.ml.nn import NeuralNetwork as NN

with open(BEATS_BLSTM[0], 'rb') as f:
    model = pickle.load(f, encoding = "latin1")

In [5]:
model.layers

In [86]:
model

In [69]:
def setWeights(t: MyMadmom, weights: dict, bidir: bool = True):
    def _getWeights(prefix: str):
        W = torch.from_numpy(np.concatenate([
                weights[f"{prefix}_input_gate_weights"].T,
                weights[f"{prefix}_forget_gate_weights"].T,
                weights[f"{prefix}_cell_weights"].T,
                weights[f"{prefix}_output_gate_weights"].T
            ])).float()
        R = torch.from_numpy(np.concatenate([
            weights[f"{prefix}_input_gate_recurrent_weights"].T,
            weights[f"{prefix}_forget_gate_recurrent_weights"].T,
            weights[f"{prefix}_cell_recurrent_weights"].T,
            weights[f"{prefix}_output_gate_recurrent_weights"].T
        ])).float()
        b = torch.from_numpy(np.concatenate([
            weights[f"{prefix}_input_gate_bias"].T,
            weights[f"{prefix}_forget_gate_bias"].T,
            weights[f"{prefix}_cell_bias"].T,
            weights[f"{prefix}_output_gate_bias"].T
        ])).float()

        return W, R, b

    def _setLSTMweights(lstmLayer: nn.LSTM, layerID: int):
        prefix = f"bidir{layerID}_fwd_layer" if bidir else f"lstm{layerID}"
        W, R, b = _getWeights(prefix)

        lstmLayer.weight_hh_l0.data = R
        lstmLayer.weight_ih_l0.data = W
        lstmLayer.bias_ih_l0.data   = b
        lstmLayer.bias_hh_l0.data   = torch.zeros_like(b)

        if bidir:
            W, R, b = _getWeights(f"bidir{layerID}_bwd_layer")

            lstmLayer.weight_hh_l0_reverse.data = R
            lstmLayer.weight_ih_l0_reverse.data = W
            lstmLayer.bias_ih_l0_reverse.data   = b
            lstmLayer.bias_hh_l0_reverse.data   = torch.zeros_like(b)

    _setLSTMweights(t.lstm1, 0)
    _setLSTMweights(t.lstm2, 1)
    _setLSTMweights(t.lstm3, 2)
    
    t.ff.weight.data = torch.from_numpy(weights["forward3_weights"].T).float()
    t.ff.bias.data   = torch.from_numpy(weights["forward3_bias"].T).float()


In [ ]:
torchModel = MyMadmom(input_size = inputSize, hidden_size = hiddenSize, output_size = outputSize, bidirectional = bidirectional)
madmomWeights = extractMadmomWeights(model, bidir = bidirectional)
setWeights(torchModel, madmomWeights, bidir = bidirectional)

#### Set the weights to the model, now checking if it works the same way

In [71]:
from madmom.audio.stft import ShortTimeFourierTransformProcessor
from madmom.audio.signal import SignalProcessor, FramedSignalProcessor
from madmom.audio.spectrogram import FilteredSpectrogramProcessor, LogarithmicSpectrogramProcessor, SpectrogramDifferenceProcessor
from madmom.ml.nn import NeuralNetworkEnsemble
from madmom.processors import ParallelProcessor, SequentialProcessor

In [73]:
frameSizes = [1024, 2048, 4096]
numBands   = 6

sig = SignalProcessor(num_channels=1, sample_rate=44100)
multi = ParallelProcessor([])
for frameSize in frameSizes:
    frames = FramedSignalProcessor(frame_size = frameSize)
    stft = ShortTimeFourierTransformProcessor() 
    filt = FilteredSpectrogramProcessor(num_bands = numBands, fmin = 30, fmax = 17000, norm_filters = True)
    spec = LogarithmicSpectrogramProcessor(mul = 1, add = 1)
    diff = SpectrogramDifferenceProcessor(diff_ratio = 0.5, positive_diffs = True, stack_diffs = np.hstack)
    # process each frame size with spec and diff sequentially
    multi.append(SequentialProcessor((frames, stft, filt, spec, diff)))
# stack the features and processes everything sequentially
proc = SequentialProcessor((sig, multi, np.hstack))

In [74]:
prepped = proc(songPath.__str__())

In [ ]:
out = model.layers[0](prepped)
modelOutputs = {0: f"{prepped.shape} -> {out.shape}"}
for i, layer in enumerate(model.layers[1:]):
    oldShape = out.shape
    out = layer(out)
    modelOutputs[i + 1] = f"{oldShape} -> {out.shape}"

modelOutputs

{0: '(3000, 266) -> (3000, 50)',
 1: '(3000, 50) -> (3000, 50)',
 2: '(3000, 50) -> (3000, 50)',
 3: '(3000, 50) -> (3000, 1)'}

In [76]:
a = model(prepped)
a.shape

(3000,)

In [77]:
b = torchModel(torch.from_numpy(prepped).float())
b.shape

torch.Size([3000, 1])

In [78]:
np.abs((b.flatten().detach().numpy() - a)).mean()

0.027866237

In [ ]:
a1 = model.layers[0](prepped)
a1.shape

(3000, 50)

In [80]:
b1 = torchModel.lstm1(torch.from_numpy(prepped).float())[0]
b1.shape
# torch.zeros(2, 25)

torch.Size([3000, 50])